## Connect to template

In `server.py`, set -
```
add_routes(app, chain_ext, path="/rag_ollama_multi_query")
```

In [24]:
from langserve.client import RemoteRunnable
import os
import json

def read_jsonl_files(folder_path):
    # 检查文件夹路径是否存在
    
    # 存储所有JSON对象的列表
    json_objects = []

    with open(folder_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 解析JSON对象并添加到列表中
            json_objects.append(json.loads(line.strip()))
            

    return json_objects

# 文件夹路径
folder_path = 'data/multifieldqa_zh.jsonl'

# 读取JSONL文件
json_objects = read_jsonl_files(folder_path)

# 打印读取的JSON对象
for obj in json_objects:
    print(obj)
    print(obj.keys())
    print(obj['input'])
    print(obj['answers'])
    print(obj['length'])
    print(obj['context'])
    break
# rag_app_ollama = RemoteRunnable("http://127.0.0.1:8000/rag-ollama-multi-query")

# rag_app_ollama.invoke("What are the different types of agent memory?")

FileNotFoundError: [Errno 2] No such file or directory: 'data/multifieldqa_zh.jsonl'

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.chat_models import ChatOllama, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader,JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langserve.client import RemoteRunnable
from DynamicSizeDynamicStepSplitter import DynamicSizeDynamicStepSplitter
from DynamicSizeFixedStepSplitter import DynamicSizeFixedStepSplitter
from FixedSizeFixedStepSplitter import FiexedSizeFixedStepSplitter
import os
import json
import chromadb
output_folder='/Users/wangyantong/Desktop/SW-RAG/packages/rag-ollama-multi-query/data/multifieldqa_en'
os.environ["OPENAI API_KEY"]="sk-53KBWArZS6U1KmNA9ZExT3BLbkFJ7AVnUypvLIDtnsIhoy9P"
for i in range(10,len(os.listdir(output_folder))):
    print(i)
    filePath=f'/Users/wangyantong/Desktop/SW-RAG/packages/rag-ollama-multi-query/data/multifieldqa_en/output_{i}.json'
    with open(filePath, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
        
    # if(i!=0):
    #     client = chromadb.Client() 
    #     client.delete_collection('rag-private')
        # client = chromadb.Client() 
        # client.delete_collection('rag-private')
        loader = JSONLoader(filePath,jq_schema='.context')
        data = loader.load()
        
        text_splitter = DynamicSizeDynamicStepSplitter(window_size=512, step_size=2)
        all_splits = text_splitter.split_documents(data)
        # Add to vectorDB
        vectorstore = Chroma.from_documents(
            documents=all_splits,
            collection_name="rag-private",
            embedding=OpenAIEmbeddings(),
        )


        QUERY_PROMPT = PromptTemplate(
            input_variables=["question"],
            template="""You are an AI language model assistant. Your task is to generate five
            different versions of the given user question to retrieve relevant documents from
            a vector database. By generating multiple perspectives on the user question, your
            goal is to help the user overcome some of the limitations of the distance-based
            similarity search. Provide these alternative questions separated by newlines.
            Original question: {question}""",
        )

        # Add the LLM downloaded from Ollama
        ollama_llm = "llama3"
        llm = ChatOllama(model=ollama_llm)

        # Run
        retriever = MultiQueryRetriever.from_llm(
            vectorstore.as_retriever(), llm, prompt=QUERY_PROMPT
        )  # "lines" is the key (attribute name) of the parsed output

        # RAG prompt
        template = """Answer the question based only on the following context:
        {context}
        Question: {question}
        """
        prompt = ChatPromptTemplate.from_template(template)

        # RAG
        model = ChatOpenAI()
        chain = (
            RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
            | prompt
            | model
            | StrOutputParser()
        )


    # Add typing for input
        class Question(BaseModel):
            __root__: str

        chain = chain.with_types(input_type=Question)
    
        o={}
        print(retriever.invoke(json_data['input']))
        o['answer']=chain.invoke(json_data['input'])
        print(json_data['answers'])
       
        output_file=f'/Users/wangyantong/Desktop/SW-RAG/packages/rag-ollama-multi-query/data/en_answer_DDS/aoutput_{i}.json'
        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(o, file, ensure_ascii=False, indent=4)
    


10
[Document(page_content='Paper Info\n\nTitle: Bistability between π-diradical open-shell and closed-shell states in indeno[1,2-a]fluorene\nPublish Date: Unkown\nAuthor List: Shantanu Mishra (from IBM Research Europe -Zurich), Manuel Vilas-Varela (from Department of Organic Chemistry, Center for Research in Biological Chemistry and Molecular Materials (CiQUS), University of Santiago de Compostela), Leonard-Alexander Lieske (from IBM Research Europe -Zurich), Ricardo Ortiz (from Donostia International Physics Center (DIPC)), Igor Rončević (from Department of Chemistry, University of Oxford), Florian Albrecht (from IBM Research Europe -Zurich), Diego Peña (from Department of Organic Chemistry, Center for Research in Biological Chemistry and Molecular Materials (CiQUS), University of Santiago de Compostela), Leo Gross (from IBM Research Europe -Zurich)', metadata={'seq_num': 1, 'source': '/Users/wangyantong/Desktop/SW-RAG/packages/rag-ollama-multi-query/data/multifieldqa_en/output_10.jso

KeyboardInterrupt: 

In [29]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_community.chat_models import ChatOllama, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader,JSONLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langserve.client import RemoteRunnable
from DynamicSizeDynamicStepSplitter import DynamicSizeDynamicStepSplitter
from DynamicSizeFixedStepSplitter import DynamicSizeFixedStepSplitter
from FixedSizeFixedStepSplitter import FiexedSizeFixedStepSplitter
import os
import json
import chromadb
output_folder='/Users/wangyantong/Desktop/SW-RAG/packages/rag-ollama-multi-query/data/multifieldqa_zh'
os.environ["OPENAI API_KEY"]="sk-53KBWArZS6U1KmNA9ZExT3BLbkFJ7AVnUypvLIDtnsIhoy9P"

for i in range(180,len(os.listdir(output_folder))):
    print(i)
    filePath=f'/Users/wangyantong/Desktop/SW-RAG/packages/rag-ollama-multi-query/data/multifieldqa_zh/output_{i}.json'
    with open(filePath, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
        
    # if(i!=0):
    #     client = chromadb.Client() 
    #     client.delete_collection('rag-private')
        client = chromadb.Client() 
        client.delete_collection('rag-private')
        loader = JSONLoader(filePath,jq_schema='.context')
        data = loader.load()
        text_splitter = DynamicSizeDynamicStepSplitter(window_size=512, step_size=2)
        all_splits = text_splitter.split_documents(data)
       
        # Add to vectorDB
        vectorstore = Chroma.from_documents(
            documents=all_splits,
            collection_name="rag-private",
            embedding=OpenAIEmbeddings(),
        )
        retriever = vectorstore.as_retriever()

        # Prompt
        # Optionally, pull from the Hub
        # from langchain import hub
        # prompt = hub.pull("rlm/rag-prompt")
        # Or, define your own:
        template = """Answer the question based only on the following context:
        {context}

        Question: {question}
        """
        
        prompt = ChatPromptTemplate.from_template(template)

        # Add the LLM downloaded from Ollama
        ollama_llm = "llama3"
        llm = ChatOllama(model=ollama_llm)

        # Run
        retriever = vectorstore.as_retriever()
        # RAG prompt
        template = """Answer the question based only on the following context:
        {context}
        Question: {question}
        """
        prompt = ChatPromptTemplate.from_template(template)

        # RAG
        model = ChatOpenAI()
        chain = (
            RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
            | prompt
            | model
            | StrOutputParser()
        )



    # Add typing for input
        class Question(BaseModel):
            __root__: str

        chain = chain.with_types(input_type=Question)

    
        o={}
        try:
            o['answer']=chain.invoke(json_data['input'])
            print(json_data['answers'])
            output_file=f'/Users/wangyantong/Desktop/SW-RAG/packages/rag-ollama-multi-query/data/zh_answer_DDS_d/aoutput_{i}.json'
            with open(output_file, 'w', encoding='utf-8') as file:
                json.dump(o, file, ensure_ascii=False, indent=4)

        except Exception:
            print("error:",i)

#3 17 33 

60
['平等兼愛、力行和節約。']
61
['3.2万人。']
62
['事故发生的日期是2012年6月2日。']
63


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


error: 63
64
['提供总览企业档案管理概况、掌握企业档案借阅流向等功能。']
65
['是的，陈瑜认为自己已经尽到了协调停车位的义务。']
66


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


['古人常用天象来预测人事，把天象与人间的吉凶祸福联系起来，如：彗星的出现被认为是预兆兵灾的凶象；月亮的盈亏圆缺被比附于人事之聚散离合。古代占星家为了用天象变化来占卜人间的吉凶祸福，还将天上星空区域与地上的州国互相对应，称作“分野”。']
67
['访问网站：https://imagescanner.fujitsu.com/r/ctl/21。']
68
['2013年新增绿色食品基地的面积达到30万亩。']
69
['是的。']
70
['全年为企业减少地方税费180亿元。']
71
['刘长生受伤的原因是在工作期间，因木料枝疤飞出击中左眼。']
72
['27天。']
73


...

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


['GDP增长31.4%,达到360亿元。财政收入增长39.9%，达到40亿元。城镇居民人均可支配收入8500元，增长18%，农牧民人均纯收入4000元，增长29.4%。全社会固定资产投资增长85.9%，达到252亿元。']
178


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


error: 178
179
['5.4%。']
180


APIConnectionError: Connection error.